In [ ]:
!pip3 install opencv-python

In [ ]:
# Import necessary libraries
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import requests
from io import BytesIO
from PIL import Image

# Define the directories
calculator_dir = 'Calculator'
mobile_dir = 'Mobile'

# Image size
img_size = 64

# Function to load images and labels
def load_images_and_labels():
    images = []
    labels = []
    
    for label, directory in enumerate([calculator_dir, mobile_dir]):
        for filename in os.listdir(directory):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (img_size, img_size))
            images.append(img)
            labels.append(label)
    
    return np.array(images), np.array(labels)

# Load images and labels
images, labels = load_images_and_labels()
labels = to_categorical(labels, num_classes=2)

# Normalize the images
images = images / 255.0

# Split data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Build the CNN model
def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = build_model()

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_val, y_val))

# Save the model
os.makedirs('Models', exist_ok=True)
model.save('Models/image_classifier.h5')

# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Inference section
def predict_image_from_url(url):
    # response = requests.get(url)
    # img = Image.open(BytesIO(response.content))
    img=Image.open(url)
    img = img.resize((img_size, img_size))
    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    model = load_model('Models/image_classifier.h5')
    predictions = model.predict(img_array)
    labels = ['Calculator', 'Mobile']
    predicted_label = labels[np.argmax(predictions)]
    
    plt.imshow(img)
    plt.title(f'Prediction: {predicted_label}')
    plt.axis('off')
    plt.show()

# Example URL for testing
url = 'https://example.com/path/to/your/image.jpg'
predict_image_from_url(url)
